In [ ]:
from glass.ete.otol.tools import module_osmtags

tags = module_osmtags(2)

In [ ]:
tags[tags.name == 'roads'].lulc_id.unique()

In [ ]:
tags.name.unique()

In [ ]:
tags[
    (tags['value'] == 'yes') &
    (tags['key'] == 'building')
]

In [ ]:
from glass.ete.otol.pg import osm_to_lulc
from glass.prop.prj import get_epsg
import datetime
import os

osm = '/mnt/d/osm2lulc02/osmdata/osm_terceira.xml'

nomenclature = 'clc'

refraster = '/mnt/d/osm2lulc02/lmt/rst_terceira.tif'

result = '/mnt/d/osm2lulc02/lulc_fmosm/terceira.gpkg'

db = '/mnt/d/osm2lulc02/lulc_fmosm/terceira_26062023.sql'

ws = '/mnt/d/osm2lulc02/debug'

time_a = datetime.datetime.now().replace(microsecond=0)
res, logs = osm_to_lulc(
    osm, nomenclature, get_epsg(refraster),
    result, savedb=db, tmpfld=ws
)
time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
q = (
    "SELECT lulc, (ST_Dump(ST_UnaryUnion(ST_Collect(geometry)))).geom AS geometry FROM ("
        "SELECT ogc_fid, 'area_upper' AS osm_module, 36 AS lulc, "
        #"CASE WHEN ST_IsValid(ST_Transform(wkb_geometry, 5015)) THEN ST_Transform(wkb_geometry, 5015) "
        #"ELSE ST_MakeValid(ST_Transform(wkb_geometry, 5015)) END AS geometry "
        "ST_Transform(wkb_geometry, 5015) AS geometry "
        "FROM multipolygons AS tcls "
        "WHERE (tcls.landuse='forest') AND ST_Area(ST_Transform(wkb_geometry, 5015)) > 10000 "
        "UNION ALL "
        "SELECT ogc_fid, 'area_upper' AS osm_module, 67 AS lulc, "
        #"CASE WHEN ST_IsValid(ST_Transform(wkb_geometry, 5015)) THEN ST_Transform(wkb_geometry, 5015) ELSE "
        #"ST_MakeValid(ST_Transform(wkb_geometry, 5015)) END AS geometry "
        "ST_Transform(wkb_geometry, 5015) AS geometry "
        "FROM multipolygons AS tcls WHERE (tcls.natural='grassland' OR tcls.landuse='grass' OR tcls.landuse='meadow') "
        "AND ST_Area(ST_Transform(wkb_geometry, 5015)) > 10000"
    ") AS mtbl GROUP BY lulc"
)

print(q)

In [ ]:
from glass.it.shp import dbtbl_to_shp
tst = dbtbl_to_shp('osm_terceira', q, 'geometry', '/mnt/d/osm2lulc02/debug/teste6.gpkg', tableIsQuery=True, olyr='rule3_4', api='ogr2ogr')

In [ ]:
from glass.ete.osm2lulc.grs import osm_to_lulc
from glass.ete.osm2lulc.utils import record_time_consumed
import datetime
import os
from glass.pys.oss import fprop, mkdir
from glass.pys.tm import now_as_str

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ws = mkdir(os.path.join(
    os.path.dirname(osm),
    fprop(result, 'fn')
), overwrite=True)

shp, time_check = osm_to_lulc(
    osm, nomenclature, refraster,
    result, tmpfld=ws,
    overwrite=True, savedb=db
)

time_b = datetime.datetime.now().replace(microsecond=0)
    
# Record time consumed in xlsx table
record_time_consumed(time_check, os.path.join(
    os.path.dirname(result),
    fprop(result, 'fn') + '.xlsx'
))

print(time_b - time_a)